# Validate and Publish Backend Flow

This notebook validates the FastAPI backend's validation and publishing steps for a LinkedIn post.

It will:
- Find an existing draft or create a new one (`POST /posts/generate`).
- Validate the post (`POST /posts/{id}/validate`).
- Publish the post (`POST /posts/{id}/publish`).

Assumes the API is running at `http://localhost:4000`. If you set an `API_KEY` in `.env`, the notebook will include it automatically from your environment.

In [ ]:
import os
import requests
from pprint import pprint

BASE_URL = os.getenv('LPG_BASE_URL', 'http://localhost:4000')
API_KEY = os.getenv('API_KEY', '')
HEADERS = { 'Content-Type': 'application/json' }
if API_KEY:
    HEADERS['x-api-key'] = API_KEY

def show(title, obj):
    print(f'\n=== {title} ===')
    pprint(obj)

print('BASE_URL =', BASE_URL)
print('Using API_KEY =', 'yes' if API_KEY else 'no')


## 1) Health check
Ensure the API is reachable.

In [ ]:
r = requests.get(f'{BASE_URL}/health', headers=HEADERS, timeout=10)
r.raise_for_status()
show('Health', r.json())


## 2) Find or create a draft
We first try to find a draft via `GET /posts?status=draft`. If none exist, we create one via `POST /posts/generate`.

In [ ]:
# Try to find an existing draft
r = requests.get(f'{BASE_URL}/posts', params={'status':'draft'}, headers=HEADERS, timeout=20)
r.raise_for_status()
drafts = r.json()
if drafts:
    draft = drafts[0]
else:
    # Create a new draft
    payload = {}  # You can set a topic: {'topic': 'AI productivity'}
    r = requests.post(f'{BASE_URL}/posts/generate', json=payload, headers=HEADERS, timeout=120)
    r.raise_for_status()
    draft = r.json()

show('Draft', draft)
post_id = draft.get('id')
assert post_id, 'No id on draft'
assert draft.get('status') == 'draft', f"Expected draft, got {draft.get('status')}"


## 3) Validate the post
Call `POST /posts/{id}/validate`. This should set `status` to `validated` and set `validatedAt`.

In [ ]:
r = requests.post(f'{BASE_URL}/posts/{post_id}/validate', headers=HEADERS, timeout=20)
r.raise_for_status()
validated = r.json()
show('Validated', validated)
assert validated.get('status') == 'validated', f"Expected validated, got {validated.get('status')}"
assert validated.get('validatedAt'), 'validatedAt missing'


## 4) Publish the post
Call `POST /posts/{id}/publish`.

- If LinkedIn credentials are configured, the backend will publish to LinkedIn and return the real URL.
- Otherwise, it returns a stub URL and still marks the post as `posted`.

In [ ]:
r = requests.post(f'{BASE_URL}/posts/{post_id}/publish', headers=HEADERS, timeout=60)
r.raise_for_status()
posted = r.json()
show('Posted', posted)
assert posted.get('status') == 'posted', f"Expected posted, got {posted.get('status')}"
assert posted.get('postedAt'), 'postedAt missing'
# linkedinPostUrl may be stubbed or real
assert posted.get('linkedinPostUrl'), 'linkedinPostUrl missing'


---
### Notes
- To test a fully real publish, set `LINKEDIN_ACCESS_TOKEN` and an author or organization URN in `.env`.
- Without credentials, the backend uses a deterministic stub URL and still sets `status` to `posted`.
- You can re-run cells to validate/publish another draft.